https://www.oreilly.com/learning/algorithmic-trading-in-less-than-100-lines-of-python-code
https://dannyvankooten.com/backtesting-12-month-sma-investing-strategy-pandas/


In [ ]:
https://dannyvankooten.com/backtesting-12-month-sma-investing-strategy-pandas/


In [34]:
import requests
from datetime import date
import json
# [oanda]
instrument = 'XAU_USD'
instrument = 'NATGAS_USD'
instrument = 'WTICO_USD'

access_token = '00f206dcbcca268040de902e9da10ea5-85f7af8ad9febfc042ffa45ff55229ce'
OANDA_PRACTICE_URL = 'https://api-fxpractice.oanda.com'

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % access_token,
}
todays_date = date.today()  # retreived in YYYY-MM-DD format
datetime_str = todays_date.strftime('%Y-%m-%d')

# https://developer.oanda.com/rest-live-v20/instrument-ep/
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
params = (
    # ('price', 'MBA'),
    ('price', 'M'),
    # ('from', '2018-08-01T15:00:00.000000000Z'),
    ('to', '%sT01:00:00.000000000Z' % datetime_str),
    # ('granularity', 'M1'),
    ('granularity', 'H1'),
    #('granularity', 'D'),
    ('count', '5000') # default = 500, max = 5000
)

response = requests.get('%s/v3/instruments/%s/candles' % (OANDA_PRACTICE_URL, instrument), headers=headers, params=params)
if response.status_code != 200:
    print("Unable to retrieve response", response)
    raise Exception("Unable to retrieve response from OANDA")

data = json.loads(response.content.decode())


In [4]:
import pickle
import pandas as pd
import math
import backtrader as bt
import backtrader.feeds as btfeeds
from collections import OrderedDict

instrument = "SPX500_USD"
import os
with open(os.getcwd() + "/trader-platform/trader-platform-modules/tests/" + instrument+ ".pk", 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data).set_index('time')  # 8
df.index = pd.DatetimeIndex(df.index)  # 9

o = []
h = []
l = []
c = []

for index, item in df.iterrows():
    ohlc = df['mid'][index]
    o.append(  float(ohlc['o']) )
    h.append(  float(ohlc['h']) )
    l.append(  float(ohlc['l']) )
    c.append(  float(ohlc['c']) ) 

df['Open'] = o
df['High'] = h
df['Low']  = l
df['Close'] = c

df.rename(columns={"volume": "Volume"}, inplace = True)
df.drop(columns=['complete', 'mid'], inplace = True)
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

#print(df.info())
print(df.head())

                      Open   High    Low  Close  Volume
time                                                   
2003-03-23 22:00:00  882.7  883.2  861.9  864.4     823
2003-03-24 22:00:00  863.4  880.4  857.6  874.2     815
2003-03-25 22:00:00  874.4  876.7  866.6  869.6     791
2003-03-26 22:00:00  867.6  874.9  858.1  868.9     770
2003-03-27 22:00:00  870.1  870.4  860.8  863.9     787


In [ ]:

class SLTPTracking(bt.Observer):
    lines = ('stop', 'take')

    plotinfo = dict(plot=True, subplot=False)

    plotlines = dict(stop=dict(ls=':', linewidth=1.5),
                     take=dict(ls=':', linewidth=1.5))

    def next(self):
        if self._owner.sl_price != 0.0:
            self.lines.stop[0] = self._owner.sl_price

        if self._owner.tp_price != 0.0:
            self.lines.take[0] = self._owner.tp_price


class PriceMomentum(bt.Strategy):
    '''
    Entry rules:
        1. Go long  if:
            a. Current bar's close is greater than X bar ago and close price crosses over lowest bollinger band
        2. Sell short if:
            a. Current bar's close is lower than X bar ago and close price crosses under highest bollinger band

    Channel Breakout exit rules:

    Setup:
      Bollinger Bands:
        - lenght: 20
        - dev standard: 1
      X: 10

    Order Execution Type:
      - Market
    Notes:
    '''
    params = (
        ('printlog', True),        
        ("period", 20),
        ("devfactor", 1),
        ("atr",2),
        ("channel_lenght", 10)
    )

    def __init__(self):

        self.order = None
        # Add a ATR indicator for volatility measure
        self.atr = bt.indicators.ATR(self.datas[0], period=20)

        # Bollinger Bands
        self.boll = bt.indicators.BollingerBands(period=self.p.period, devfactor=self.p.devfactor)
        
        self.cross_down = bt.indicators.CrossDown(self.data.close, self.boll.lines.top)
        self.cross_up = bt.indicators.CrossUp(self.data.close, self.boll.lines.bot)
        
        self.cross_down.plotinfo.plotskip=True
        self.cross_up.plotinfo.plotskip=True

        self.sl_price, self.tp_price = 0.0, 0.0
        # an array where each operation (buy/sell) will be stored for further analysis
        self.orders_tracking = OrderedDict()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            ep = order.executed.price
            es = order.executed.size
            tv = ep * es
            leverage = self.broker.getcommissioninfo(self.data).get_leverage()
            margin = abs((ep * es) / leverage)


            if order.isbuy():
                self.log(
                    '%s - BUY EXECUTED on %s, Price: %.2f, Size: %.2f, Comm %.2f, Margin %.2f' %
                    (self.data0._name,
                     order.info['name'],
                     order.executed.price,
                     order.executed.size,
                     order.executed.comm,
                     margin))

            else:  # Sell
                self.log(
                    '%s - SELL EXECUTED on %s, Price: %.2f, Size: %.2f, Comm %.2f, Margin %.2f' %
                    (self.data0._name,
                        order.info['name'],
                     order.executed.price,
                     order.executed.size,
                     order.executed.comm,
                     margin))

            # track all operation
            obj = (
                self.datas[0].datetime.date(0).isoformat(),
                "BUY" if order.isbuy() else "SELL",
                order.info['name'],
                order.executed.price,
                order.executed.value,
                order.executed.comm,
                order.size
            )
            #self.orders_tracking[self.datas[0].datetime.date(0).isoformat()] = obj

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.data.close[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        if not self.position:
            self.sl_price = math.nan
            if self.data.close > self.data.close[-self.p.channel_lenght] and self.cross_up:
 
                #self.sell(exectype=bt.Order.Stop, price=self.boll.lines.top[0], size=self.p.size)
                self.order = self.buy()
                self.order.addinfo(name="Market Order")

 
            if self.data.close < self.data.close[-self.p.channel_lenght] and self.cross_down:
                #self.buy(exectype=bt.Order.Stop, price=self.boll.lines.bot[0], size=self.p.size)
                self.order = self.sell()
                self.order.addinfo(name="Market Order")

 
        else:
            
            if self.position.size > 0:
                if self.data_close[-1] < self.boll.lines.top[-1] and self.data_close[0] > self.boll.lines.top[0]:
                    self.order = self.close()
                    self.order.addinfo(name="Market Order - Close Long")
                    
            if self.position.size < 0:
                if self.data_close[-1] > self.boll.lines.bot[-1] and self.data_close[0] < self.boll.lines.bot[0]:
                    self.order = self.close()
                    self.order.addinfo(name="Market Order - Close Short")

            # Stop 1 - fixed stop loss based on volatility measure
            if self.position.size > 0:
                stop_price = self.position.price - self.p.atr * self.atr[-1]
                if self.data.close[0] < stop_price:  # stop loss in case of LONG
                    self.order = self.close()
                    self.order.addinfo(name="Stop Loss Long")
            elif self.position.size < 0:
                stop_price = self.position.price + self.p.atr * self.atr[-1]
                if self.data.close[0] > stop_price:  # stop loss in case of LONG
                    self.order = self.close()
                    self.order.addinfo(name="Stop Loss Short")

            self.sl_price = stop_price
 
 
        if self.p.printlog and 0:
            print('---------------------------- NEXT ----------------------------------')
            print("1: Data Name:                            {}".format(instrument))
            print("2: Bar Num:                              {}".format(len(self.data)))
            #print("3: Current date:                         {}".format(data.datetime.date(0).isoformat()   ))
            print('4: Open:                                 {}'.format(self.data.open[0]))
            print('5: High:                                 {}'.format(self.data.high[0]))
            print('6: Low:                                  {}'.format(self.data.low[0]))
            print('7: Close:                                {}'.format(self.data.close[0]))
            print('8: Volume:                               {}'.format(self.data.volume[0]))
            print('9: Position Size:                       {}'.format(self.position.size))
            print('--------------------------------------------------------------------')
 
    def notify_trade(self,trade):
        if trade.isclosed and self.p.printlog is True:
            dt = self.data.datetime.date()
 
            print('---------------------------- TRADE ---------------------------------')
            print("1: Data Name:                            {}".format(instrument))
            print("2: Bar Num:                              {}".format(len(trade.data)))
            print("3: Current date:                         {}".format(dt))
            print('4: Status:                               Trade Complete')
            print('5: Ref:                                  {}'.format(trade.ref))
            print('6: PnL:                                  {}'.format(round(trade.pnl,2)))
            print('--------------------------------------------------------------------')


    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))


import numpy
datafeed = btfeeds.PandasData(dataname=df)

channel_lengths  = numpy.arange(5, 30, 5, dtype=int).tolist()

for channel_length in channel_lengths:    
    # Create an instance of cerebro
    cerebro = bt.Cerebro(stdstats=True)
    # Set our desired cash start
    startcash = 1000
    cerebro.broker.setcash(startcash)
    
    # https://www.backtrader.com/docu/commission-schemes/commission-schemes/
    # https://backtest-rookies.com/2017/09/26/backtrader-commission-schemes/
    value_per_tick = 0.01  # value per tick see Trading Plan tables
    decimal_digit_number = 2  # number of decimal digit after point
    cerebro.broker.setcommission(commission=2, margin=65,mult=value_per_tick / (10 ** -decimal_digit_number))
    
    # Analyzer
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='dd')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
    
    cerebro.addanalyzer(bt.analyzers.SharpeRatio,  _name="mySharpe",  riskfreerate=0.01,     timeframe=bt.TimeFrame.Months)
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="myReturn")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="mySqn")
    
    cerebro.addobserver(SLTPTracking)
    
    # Add the strategy
    cerebro.addstrategy(PriceMomentum,printlog=False,atr=2,
                        channel_lenght=channel_length, devfactor=1, period=20 )
    
    # Add the data to Cerebro
    cerebro.adddata(datafeed,name=instrument)
    
    # Run over everything
    strategies = cerebro.run()
    #first_strategy = strategies[0]
    
    #Get final portfolio Value
    portvalue = cerebro.broker.getvalue()
    pnl = portvalue - startcash
     
    #Print out the final result
    #print('Final Portfolio Value: ${}'.format(round(portvalue,2)))
    print('P/L: ${}'.format(round(pnl,2)))
 


P/L: $-177.7

P/L: $-71.48

P/L: $-36.12

P/L: $-60.53

P/L: $-91.92

In [ ]:
# Finally plot the end results
cerebro.plot(iplot=False, style='candlestick')